<a href="https://colab.research.google.com/github/luciabarg/mentoria_img/blob/master/4-Aprendizaje_Supervisado/5_Generaci%C3%B3n_de_fotos_recortadas_y_tabla_An%C3%A1lisis_y_Curaci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center>
<h4>Universidad Nacional de Córdoba - Facultad de Matemática, Astronomía, Física y Computación </h4>
<h3>Diplomatura en Ciencia de Datos, Aprendizaje Automático y sus Aplicaciones</h3>
<h2>Mentoría: Detección de objetos en imágenes</h2>
</center>


En la siguiente notebook, incorporaremos parte de lo realizado en las entregas 1 de los grupos de estudiantes: 

* [Grupo 1](https://github.com/Ana1890/Deteccion-de-objetos-en-imagenes) : Ana Rusconi, Ignacio Grosso y  Mariano Lucero

* [Grupo 2](https://github.com/leorodriguez/mentoria-diplodatos-grupo2):  Diego Gonzalez, Florencia Caballero, Joel Vivas y Leonardo Rodriguez


#### <a name="generacion_tabla"></a>
#### **Generación imágenes recortadas y tabla de features**

En las entregas anteriores tuvieron la oportunidad de explorar una pequeña muestra de las imágenes para que el análisis sea mas representativo del conjunto de datos total, en esta entrega utilizaremos el dataset completo.

Esta notebook es para que vean como se la genera a las imágenes y la tabla que utilizarán en la siguiente notebook '[6-Aprendizaje Supervisado](https://colab.research.google.com/drive/1tVQh9u-HdyqQprkWqGoU1jikswD_Xq5d?usp=sharing)'

La carpeta con los datos, es compartida en el [Drive](https://drive.google.com/drive/folders/1-bl3h1g1WROMmz66A2h7t2n3leTjhYxH?usp=sharing) y en el desarrollo de esta entrega con las consignas, se utilizará la misma en el entorno de Colab, pero también estarán en el repositorio de Github de la mentoría, en la [carpeta del práctico](https://github.com/luciabarg/mentoria_img/tree/master/4-Aprendizaje%20Supervisado).

La siguiente notebook usa como entorno Colab:

In [2]:
#vinculamos el Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# librerías necesarias 
import os
from IPython.display import Image as Imge
import pandas as pd
import numpy as np
import glob

#seguimos agregando librerías necesarias:
#las que no tengan, vayan instalándolas
import cv2
import numpy as np

import matplotlib.pyplot as plt
import PIL
from PIL import Image
from os import path

from google.colab.patches import cv2_imshow #en Colab!

import seaborn as sns

import re

import json

In [10]:
#nos posicionamos en la carpeta donde se encuentra la tabla:
os.chdir('/content/drive/My Drive/Mentoria/crop')
#listamos solo las carpetas
! ls -d */
! ls -d */*/

Amarillos/  Mano-Amarillo/  Manos/  Mano-Verde/  Verdes/
Mano-Amarillo/Amarillo/  Mano-Verde/Mano/
Mano-Amarillo/Mano/	 Mano-Verde/Verde/


Para que se hagan una idea de los datos que vamos a generar, los visualizamos con una tabla:

In [11]:
#leemos el archivo de la tabla:
csv_wlabels_df = pd.read_csv('csv_wlabels_all_df.csv', index_col=0)
print(csv_wlabels_df.shape)
csv_wlabels_df.head()

(15197, 2061)


,filename,xmin,xmax,ymin,ymax,Time,Size,Type,Hand_Pixel_Width,Hand_Pixel_Height,Area,guantes,color,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,M_G_Mostrando_Amarillos_0_double_box1.jpg,218,461,464,715,M,G,train,243,251,60993,si,Amarillo,0.740974,1.066019,0.132695,0.224892,0.140456,0.129134,0.201447,1.076733,0.461108,0.299800,0.457940,0.201904,0.569677,0.737381,0.480929,0.200076,0.018410,0.674050,0.028826,0.005310,0.128693,0.592353,0.138541,0.141684,0.363294,0.264674,0.519292,...,0.463567,0.091676,0.300955,0.696112,2.409413,0.536980,0.102273,0.157441,0.165093,1.067619,0.751934,0.231443,0.487437,0.446320,0.822407,0.527523,0.217110,0.489194,1.535306,0.521390,0.599480,0.201182,0.391337,0.040206,1.347440,0.448389,0.462778,0.590751,0.877142,0.009986,0.939221,0.859877,0.981910,0.356366,1.748501,0.288037,0.079172,0.266234,0.078312,0.421612
1,M_G_Mostrando_Amarillos_0_double_box2.jpg,617,865,432,598,M,G,train,248,166,41168,si,Amarillo,1.621832,0.208859,0.627041,0.061717,0.052486,0.064176,0.479256,1.068516,0.509059,0.207385,0.218195,0.169190,0.695486,0.237641,0.146711,0.126334,0.029900,0.148363,0.126833,0.181351,0.044585,0.447270,0.134360,0.113650,1.153900,0.653832,0.041024,...,0.816892,0.027489,0.183116,0.626535,1.734605,0.672962,0.030057,0.101685,0.295544,0.392251,0.874054,0.105285,0.110713,0.421417,1.335695,0.500971,1.139208,0.110803,0.850265,0.618181,0.077352,0.376629,0.319234,0.783792,0.427963,0.133063,0.064981,0.017857,0.320706,0.219192,1.199712,0.808466,0.491182,0.356616,0.812572,0.481357,0.071471,0.572322,0.052072,0.213453
2,M_G_Mostrando_Amarillos_100_double_box1.jpg,831,1138,256,408,M,G,train,307,152,46664,si,Amarillo,1.029100,0.246964,0.825601,0.460009,0.467037,0.339540,0.653041,1.324649,0.890402,0.488183,0.639772,0.085819,0.928661,0.148958,0.066155,0.125391,0.191808,0.506981,0.257565,0.073959,0.115524,0.552251,0.764911,0.047104,0.207601,0.126297,0.286010,...,0.367434,0.436251,0.000000,0.863331,0.112969,0.902164,0.040710,0.467132,0.242757,0.853286,0.677067,0.300664,0.587456,0.112658,1.077973,0.144736,0.294853,0.065939,0.708062,1.192815,0.317312,0.421216,0.425882,1.091190,0.595093,0.138662,0.045023,0.543350,0.155850,0.305535,0.463963,0.355296,0.360880,0.322859,0.917611,0.408334,0.733588,0.416891,0.151471,0.045931
3,M_G_Mostrando_Amarillos_100_double_box2.jpg,511,812,8,228,M,G,train,301,220,66220,si,Amarillo,0.086092,0.455828,0.418580,0.131673,0.437982,0.424093,0.350489,1.974670,0.183887,0.241193,0.046988,0.312450,0.497026,0.135973,0.195146,0.010869,0.109539,0.234128,0.235778,0.108441,0.054660,0.327069,0.439913,0.050670,0.159292,0.159228,0.311495,...,0.625021,0.219692,0.029219,0.496582,1.540741,2.338270,0.039844,0.174714,0.383931,0.449525,0.104648,0.205243,0.104140,0.109657,1.092486,0.087052,0.451129,0.034648,0.272219,0.209507,0.138235,0.193764,0.081227,1.006930,1.506502,0.044992,1.047619,0.605296,0.391379,0.164156,0.325708,0.028549,0.225465,0.325055,0.460382,0.127341,0.140412,0.157423,0.296102,0.023314
4,M_G_Mostrando_Amarillos_101_double_box1.jpg,668,988,0,222,M,G,train,320,222,71040,si,Amarillo,0.068881,0.185060,0.427007,0.637546,0.357156,0.125816,0.450526,1.246901,0.236921,0.393104,0.632759,0.413947,0.119917,0.192305,0.372677,0.075668,0.103642,0.165934,0.239047,0.054987,0.129182,0.178563,0.217686,0.130172,0.080214,0.346873,0.392784,...,0.205059,0.225329,0.062677,0.558553,1.780167,1.919532,0.165511,0.649451,0.168220,0.127490,0.310012,0.071468,0.196005,0.042177,1.261848,0.216220,0.501204,0.194764,0.183244,0.426975,0.477355,0.349516,0.131819,0.489718,0.981365,0.049099,0.471963,0.190929,0.413278,0.175050,0.523058,0.028629,0.076427,0.462088,0.694069,0.060703,0.065414,0.296513,0.368063,0.039469


Tenemos 3 conjuntos de datos, Mañana, Tarde y Noche:

In [12]:
#nos posicionamos en la carpeta donde se encuentran los datos iniciales:
os.chdir('/content/drive/My Drive/Mentoria')
! ls -d Dataset-*/

Dataset-M/  Dataset-N/	Dataset-T/


In [ ]:
# Definimos el path de la carperta para usarlo luego:
data_sets_path = "/content/drive/My Drive/Mentoria"
os.chdir('{}'.format(data_sets_path))

In [ ]:
#listamos todos las imágenes del dataset:
list_dir = [i for i in glob.glob('./Dataset-*/*/*/*.jpg')]
len(list_dir)

list_jpg_files = []

for file_jpg in list_dir:
  x = re.search('[A-Z]\w+_.+jpg', file_jpg)
  list_jpg_files.append(x.group())

print(len(list_jpg_files))  # len(list_jpg_files) = 8509


8509


En lo siguiente, exploramos las carpetas para extraer los '.csv' y armar una sola tabla con toda la información. Agregamos también nuevas columnas con la información de las carpetas. 

In [ ]:
# acá exploramos todos los archivos 'csv' por carpetas
os.chdir('{}'.format(data_sets_path))

def path(day_time, size, typ):
    return ("{path}/Dataset-{time}/{size}"
    "/{time}-{size}-Labels_{typ}.csv").format(path=data_sets_path,
                                              time=day_time, size=size, typ=typ)
def build_df(day_time, size, typ):
    ds = pd.read_csv(path(day_time, size, typ))
    ds['Time'] = day_time
    ds['Size'] = size
    ds['Type'] = typ
    return ds


types = ["train", "test"]
day_time_list = ["M", "T", "N"]
size_list = ["G", "P"]
frames = [build_df(t, s, typ) for t in day_time_list for s in size_list for typ in types]
total_data = pd.concat(frames, ignore_index=True)
total_data["Hand_Pixel_Width"]  = (total_data["xmax"] - total_data["xmin"])
total_data["Hand_Pixel_Height"] = (total_data["ymax"] - total_data["ymin"])
total_data["Area"] = total_data["Hand_Pixel_Width"] * total_data["Hand_Pixel_Height"]
print(total_data.shape)
total_data.head()

(15203, 14)


,filename,width,height,class,xmin,ymin,xmax,ymax,Time,Size,Type,Hand_Pixel_Width,Hand_Pixel_Height,Area
0,M_G_Trabajando_Mano-Verde_7.jpg,1920,1080,hand,877,262,1121,422,M,G,train,244,160,39040
1,M_G_Trabajando_Mano-Verde_7.jpg,1920,1080,hand,1104,121,1266,278,M,G,train,162,157,25434
2,M_G_Trabajando_Verdes_170.jpg,1920,1080,hand,1001,332,1329,613,M,G,train,328,281,92168
3,M_G_Trabajando_Verdes_170.jpg,1920,1080,hand,1233,289,1353,395,M,G,train,120,106,12720
4,M_G_Trabajando_Manos_28.jpg,1920,1080,hand,812,219,944,320,M,G,train,132,101,13332


Eliminamos las columnas que no serán relevantes a nuestro análisis, tales como 'width',	'height',	'class':

In [ ]:
#eliminamos columnas innecesarias
all_df = total_data.drop(['width',	'height',	'class'], axis=1)
all_df.shape

(15203, 11)

In [ ]:
#hacemos una lista de las filas únicas del dataset
list_filename_csv = all_df.filename.unique().tolist()
len(list_filename_csv)  # len(list_filename_csv) = 8509

8509

In [ ]:
#definimos una función para ver diferencias entre listas
def diff(list1, list2):
    return list(set(list1).symmetric_difference(set(list2)))

In [ ]:
#verificamos que haya la misma cantidad de filas únicas como fotos en el dataset
diff(list_jpg_files, list_filename_csv)

[]

In [ ]:
##guardamos la lista en un archivo para no tener q generarla todas las veces:
#with open('list_jpg_all.txt', 'w') as f:
#    f.write(json.dumps(list_filename_csv))

In [ ]:
#abrimos el archivo con la lista:
with open('list_jpg_all.txt', 'r') as f:
    list_filename_csv = json.loads(f.read())
  
list_filename_csv[0:5]

['T_P_Trabajando_Manos_101.jpg',
 'T_P_Trabajando_Manos_136.jpg',
 'T_P_Trabajando_Verdes_152.jpg',
 'T_P_Trabajando_Manos_172.jpg',
 'T_P_Trabajando_Verdes_42.jpg']

Agregamos columnas que sirvan para seguir discriminando los datos:

In [ ]:
# identificamos cuantos valores de cada 'categoría' tenemos, agregamos las columnas
# 'guantes' y 'color' acorde a las mismas y las guardamos en un diccionario:
tipos = ['Manos', 'Amarillos', 'Verdes', 'Mano-Verde','Mano-Amarillo']
# recordemos que las categorías 'Mano-Verde' y 'Mano-Amarillo' luego hay q 
# determinar cual fila se corresponde a la mano y cuál al guante:

dict_jpg = {}
for tipo in tipos:
    dict_jpg[tipo] = all_df[all_df['filename'].astype(str).str.contains(tipo)].copy()
    if tipo == 'Manos':
        dict_jpg[tipo]['guantes'] = 'no'
        dict_jpg[tipo]['color'] = 'mano'
    elif tipo == 'Amarillos':
        dict_jpg[tipo]['guantes'] = 'si'
        dict_jpg[tipo]['color'] = 'Amarillo'
    elif tipo == 'Verdes':
        dict_jpg[tipo]['guantes'] = 'si'
        dict_jpg[tipo]['color'] = 'Verde'
    elif tipo == 'Mano-Verde':
        dict_jpg[tipo]['guantes'] = 'si'
        dict_jpg[tipo]['color'] = 'Verde'
    elif tipo == 'Mano-Amarillo':
        dict_jpg[tipo]['guantes'] = 'si'
        dict_jpg[tipo]['color'] = 'Amarillo'
        

In [ ]:
#contabilizamos la cantidad de filas a cada categoría:
tipos = ['Manos', 'Amarillos', 'Verdes', 'Mano-Verde','Mano-Amarillo']

data_tot = []
for tipo in tipos:
  print(tipo+': ', dict_jpg[tipo].shape[0])  
  data_tot.append(dict_jpg[tipo].shape[0])


#la suma debería coincidir con la cantidad de filas del dataset:
print('\n Total de datos:', sum(data_tot))

Manos:  4525
Amarillos:  3960
Verdes:  3857
Mano-Verde:  1420
Mano-Amarillo:  1441

 Total de datos: 15203


In [ ]:
#verificamos que sigamos teniendo la misma cantidad de filas: 
sum(data_tot) == all_df.shape[0]

True

In [ ]:
#pasaré el diccionario anterior a un DataFrame. Si se les ocurre una forma mas elegante de hacelo, bienvenido sea!!
manos_df = pd.DataFrame.from_dict(dict_jpg['Manos'])
amarillos_df = pd.DataFrame.from_dict(dict_jpg['Amarillos'])
verdes_df = pd.DataFrame.from_dict(dict_jpg['Verdes'])
mano_verde_df = pd.DataFrame.from_dict(dict_jpg['Mano-Verde'])
mano_amarillo_df = pd.DataFrame.from_dict(dict_jpg['Mano-Amarillo'])
# lista de DataFrame:
tipos_df = [manos_df, amarillos_df, verdes_df, mano_verde_df, mano_amarillo_df]

In [ ]:
# los juntamos en un solo csv y lo guardamos por única vez:
all_df_csv = pd.concat(f for f in tipos_df)  #
#all_df_csv.to_csv('all_df.csv')
all_df_csv.shape

(15203, 13)

In [ ]:
#volvemos a leer el archivo:
all_jpg_df = pd.read_csv('all_df.csv', index_col=0)
all_jpg_df.head()

,filename,xmin,ymin,xmax,ymax,Time,Size,Type,Hand_Pixel_Width,Hand_Pixel_Height,Area,guantes,color
4,M_G_Trabajando_Manos_28.jpg,812,219,944,320,M,G,train,132,101,13332,no,mano
5,M_G_Trabajando_Manos_28.jpg,987,256,1105,372,M,G,train,118,116,13688,no,mano
12,M_G_Trabajando_Manos_0.jpg,625,243,751,339,M,G,train,126,96,12096,no,mano
13,M_G_Trabajando_Manos_142.jpg,872,220,1019,367,M,G,train,147,147,21609,no,mano
14,M_G_Trabajando_Manos_142.jpg,714,213,839,343,M,G,train,125,130,16250,no,mano


Como anteriores prácticos de pre-análisis, creamos una carpeta para guardar las imágenes recortadas de la región de la mano:

In [ ]:
# las carpetas las creamos una sola vez
#! mkdir crop
#! mkdir crop/Manos
#! mkdir crop/Amarillos
#! mkdir crop/Verdes
#! mkdir crop/Mano-Verde
#! mkdir crop/Mano-Amarillo

In [ ]:
tipos = ['Manos', 'Amarillos', 'Verdes', 'Mano-Verde','Mano-Amarillo']

for tipo in tipos:
  path_chosen = 'crop/'+tipo +'/'
  print(path_chosen)

crop/Manos/
crop/Amarillos/
crop/Verdes/
crop/Mano-Verde/
crop/Mano-Amarillo/


In [ ]:
# No pude extraer del nombre de archivo el 'tipo'. Entonces, copié el archivo csv en crop, 
# hice los cortes de las imágenes y las moví luego. Si se les ocurre como automatizar lo siguiente, bienvenido!

In [ ]:
#! cp all_df.csv crop
! ls crop

all_amarillos_cropped_df.csv  Amarillos      Mano-Verde
all_df.csv		      Mano-Amarillo  Verdes
all_manos_cropped_df.csv      Manos


In [ ]:
os.chdir('/content/drive/My Drive/Mentoria')

In [ ]:
#listamos todos las imágenes del dataset:
list_dir = [i for i in glob.glob('./Dataset-*/*/*/*.jpg')]
len(list_dir)

list_jpg_files = []

for file_jpg in list_dir:
  x = re.search('[A-Z]\w+_.+jpg', file_jpg)
  list_jpg_files.append(x.group())

print(len(list_jpg_files))  # len(list_jpg_files) = 8509
#test_list = list_dir[0:5]
#test_list 

8509


In [ ]:
#bue.. además que no me salió el loop, bah, lo del loop del regex.. quizás fue mejor, así vamos tipo por tipo, va lento
#tipos = ['Manos', 'Amarillos', 'Verdes', 'Mano-Verde','Mano-Amarillo']
tipo = 'Mano-Verde'
path_chosen = 'crop/'+tipo+'/'


data_img_df = []

for file_name in list_dir:
  #print(file_name)
  try:
     filename = re.search('[A-Z]\w+_Mano-Verde_.+jpg', file_name).group()
     #print(filename)
     row_csv = all_jpg_df[all_jpg_df['filename'].str.contains(filename)]
     #print(row_csv) 

     num_mano = 1
     if (row_csv.shape[0] == num_mano):
       row = row_csv.iloc[0]
       img_row_file = row.filename
       #print(img_row_file)
       #print(row_csv)
       img  = cv2.imread(file_name)
       #cv2_imshow(img)
       xmin = row.xmin.item()
       xmax = row.xmax.item()
       ymin = row.ymin.item()
       ymax = row.ymax.item()
      
       time = row.Time
       size = row.Size
       sets = row.Type
       witdh  = row.Hand_Pixel_Width.item()
       height = row.Hand_Pixel_Height.item()	
       area = row.Area.item()
       guantes = row.guantes
       color  = row.color

       img_crop_name = img_row_file[:-4]+'_single_box.jpg'
       path = path_chosen +img_crop_name
       #print(img_crop_name)
       #print(path)
       data_img = [img_crop_name, xmin, xmax, ymin,ymax, time, size,sets,witdh,height,area,guantes,color]
       #print(data_img)
       data_img_df.append(data_img)
       img_crop = img[ymin:ymax, xmin:xmax]
       #cv2_imshow(img_crop)
       #cv2.imwrite(path,img_crop)

     else:
      row1 = row_csv.iloc[0]
      img_row1_file = row1.filename
      #print(img_row1_file)
      img_row1=cv2.imread(file_name)
      #cv2_imshow(img_row1)
      xmin_r1 = row1.xmin.item()
      xmax_r1 = row1.xmax.item()
      ymin_r1 = row1.ymin.item()
      ymax_r1 = row1.ymax.item()

      time_r1 = row1.Time
      size_r1 = row1.Size
      sets_r1 = row1.Type
      witdh_r1  = row1.Hand_Pixel_Width.item()
      height_r1 = row1.Hand_Pixel_Height.item()	
      area_r1 = row1.Area.item()
      guantes_r1 = row1.guantes
      color_r1  = row1.color

      img_crop_name_row1 = img_row1_file[:-4]+'_double_box1.jpg'
      img_crop_row1 = img_row1[ymin_r1:ymax_r1, xmin_r1:xmax_r1] 
      path1 = path_chosen+img_crop_name_row1
      #print(path1)

      data_img1 = [img_crop_name_row1, xmin_r1, xmax_r1, ymin_r1, ymax_r1, 
                  time_r1, size_r1, sets_r1, witdh_r1, height_r1, 
                  area_r1, guantes_r1, color_r1]
      #print(data_img1)
#      #cv2_imshow(img_crop_row1)
      data_img_df.append(data_img1)
      #cv2.imwrite(path1, img_crop_row1)

      row2 = row_csv.iloc[1]
      img_row2_file = row2.filename
      img_row2=cv2.imread(file_name)
    
      xmin_r2 = row2.xmin.item()
      xmax_r2 = row2.xmax.item()
      ymin_r2 = row2.ymin.item()
      ymax_r2 = row2.ymax.item()

      time_r2 = row2.Time
      size_r2 = row2.Size
      sets_r2 = row2.Type
      witdh_r2  = row2.Hand_Pixel_Width.item()
      height_r2 = row2.Hand_Pixel_Height.item()	
      area_r2 = row2.Area.item()
      guantes_r2 = row2.guantes
      color_r2  = row2.color

      img_crop_name_row2 = img_row2_file[:-4]+'_double_box2.jpg'
      
      data_img2 = [img_crop_name_row2, xmin_r2, xmax_r2, ymin_r2,ymax_r2, 
                  time_r2, size_r2,sets_r2,witdh_r2,height_r2,
                  area_r2, guantes_r2, color_r2]
      data_img_df.append(data_img2)
      #print(data_img2)

      img_crop_row2 = img_row2[ymin_r2:ymax_r2, xmin_r2:xmax_r2] 
      path2 = path_chosen+img_crop_name_row2
      #print(path2)
      #cv2_imshow(img_crop_row2)
      #cv2.imwrite(path2, img_crop_row2)

  except:
     filename = None


In [ ]:
cropped_name_df = 'crop/all_mano_verde_cropped_df.csv'
print(cropped_name_df)
cropped_df = pd.DataFrame(data_img_df,
                          columns = ['filename','xmin', 'xmax','ymin','ymax',
                                     'Time', 'Size', 'Type', 'Hand_Pixel_Width', 'Hand_Pixel_Height',
                                     'Area', 'guantes', 'color' 
                               ])  
cropped_df
print(cropped_df.shape[0])
#cropped_df.to_csv(cropped_name_df) 

#hacemos una lista de las filas únicas del dataset
list_cropped_df_csv = cropped_df.filename.unique().tolist()
len(list_cropped_df_csv) 
# debería coincidir con la cantidad del tipo que estamos cortando. .no entiendo porque perdemos 
# fotos en el camino.. si quieren buscarlas..
#Manos:  4525 - list_cropped_df_csv:  4524.... 
#Amarillos:  3960 - list_cropped_df_csv: 3958
#Verdes:  3857 - list_cropped_df_csv: 3855
#Mano-Verde:  1420 - list_cropped_df_csv: 1419
#Mano-Amarillo:  1441 - list_cropped_df_csv: 1441


crop/all_mano_verde_cropped_df.csv
1419


1419

Ya cortamos las imágenes y clasificamos a ojo las categorías 'Mano-Amarillo' y 'Mano-Verde' y generamos los siguientes csv:

In [ ]:
#nos posicionamos en la carpeta muestra:
os.chdir('/content/drive/My Drive/Mentoria/')
! ls -d Dataset-*/
os.chdir('/content/drive/My Drive/Mentoria/crop')
! ls *csv

Dataset-M/  Dataset-N/	Dataset-T/
all_Amarillos_cropped_df.csv	  all_Mano-Verde_cropped_df.csv
all_df.csv			  all_Verdes_cropped_df.csv
all_Mano-Amarillo_cropped_df.csv  Mano-Amarillo_mano_cropped_df.csv
all_Manos_cropped_df.csv


In [ ]:
#volvemos a leer los archivos:
tipos = ['Manos', 'Amarillos', 'Verdes', 'Mano-Verde','Mano-Amarillo']

for tipo in tipos:
  print(tipo)

  cropped_name_df = 'all_'+tipo +'_cropped_df.csv'
  cropped_df = pd.read_csv(cropped_name_df, index_col=0)
  list_filename_df = cropped_df.filename.unique().tolist()
  print('cantidad de fotos: ', len(list_filename_df)) 
  print(cropped_name_df,'shape:', cropped_df.shape, '\n')

Manos
cantidad de fotos:  4524
all_Manos_cropped_df.csv shape: (4524, 13) 

Amarillos
cantidad de fotos:  3958
all_Amarillos_cropped_df.csv shape: (3958, 13) 

Verdes
cantidad de fotos:  3855
all_Verdes_cropped_df.csv shape: (3855, 13) 

Mano-Verde
cantidad de fotos:  1419
all_Mano-Verde_cropped_df.csv shape: (1419, 13) 

Mano-Amarillo
cantidad de fotos:  1441
all_Mano-Amarillo_cropped_df.csv shape: (1441, 13) 



Como ya hemos visto, tenemos tres *clases*:
 'Mano', 'Amarillo' y 'Verde'

In [ ]:
# nos movemos a la carpeta crop:
os.chdir('/content/drive/My Drive/Mentoria/crop/')
! ls -d */
! ls -d */*/

Amarillos/  Mano-Amarillo/  Manos/  Mano-Verde/  Verdes/
Mano-Amarillo/Amarillo/  Mano-Verde/Mano/
Mano-Amarillo/Mano/	 Mano-Verde/Verde/


Verificamos los archivos y la cantidad de fotos en cada carpeta:

In [ ]:
#volvemos a leer los archivos:
tipos = ['Amarillos', 'Manos', 'Verdes']

for tipo in tipos:
  print(tipo)
  cropped_name_df = 'all_'+tipo +'_cropped_df.csv'
  cropped_df = pd.read_csv(cropped_name_df, index_col=0)
  list_filename_df = cropped_df.filename.unique().tolist()

  print('cantidad de fotos: ', len(list_filename_df)) 
  print(cropped_name_df,'shape:', cropped_df.shape, '\n')

Amarillos
cantidad de fotos:  3958
all_Amarillos_cropped_df.csv shape: (3958, 13) 

Manos
cantidad de fotos:  4524
all_Manos_cropped_df.csv shape: (4524, 13) 

Verdes
cantidad de fotos:  3855
all_Verdes_cropped_df.csv shape: (3855, 13) 



In [ ]:
#Para agrupar los archivos de una carpeta según su nombre, 
#os.chdir('/content/drive/My Drive/Mentoria/crop/Amarillos')            # 3958 fotos 
#os.chdir('/content/drive/My Drive/Mentoria/crop/Manos')                # 4524 fotos 
#os.chdir('/content/drive/My Drive/Mentoria/crop/Verdes')                # 3855 fotos 
extension = 'jpg'
list_cropped_class = [i for i in glob.glob('*.{}'.format(extension))]
len(list_cropped_class)

3855

In [ ]:
os.chdir('/content/drive/My Drive/Mentoria/crop/Mano-Amarillo/Amarillo')            # 3958 fotos 
! ls Mano_Verde

ls: cannot access 'Mano_Verde': No such file or directory


Para las carpetas 'Mano-Verde' y 'Mano-Amarillo':

In [ ]:
# nos movemos a la carpeta de Mano-Amarillo / Mano-Verde respectivamente:
# os.chdir('/content/drive/My Drive/Mentoria/crop/Mano-Amarillo/Mano')        # 707 fotos
# os.chdir('/content/drive/My Drive/Mentoria/crop/Mano-Amarillo/Amarillo')     # 736 fotos  

# os.chdir('/content/drive/My Drive/Mentoria/crop/Mano-Verde/Mano')           # 714 fotos
# os.chdir('/content/drive/My Drive/Mentoria/crop/Mano-Verde/Verde')          # 703 fotos



In [ ]:
#Para agrupar los archivos de una carpeta según su nombre 
extension = 'jpg'
list_cropped_class = [i for i in glob.glob('*.{}'.format(extension))]
list_cropped_class.sort()
len(list_cropped_class)

714

In [ ]:
#esto vamos cambiando respectivamente por amarillos o verdes
#list_mano_amarillo_amarillo = list_cropped_class.copy()
#len(list_mano_amarillo_amarillo)
list_mano_verde_mano = list_cropped_class.copy()
len(list_mano_verde_mano)

714

In [ ]:
##guardamos la lista en un archivo para no tener q generarla todas las veces:

#volvemos a la carpeta mano-guante
os.chdir('/content/drive/My Drive/Mentoria/crop') 
#list_data_muestra = list_mano_amarillo_amarillo
#filename = 'list_mano_amarillo_amarillo.txt'
#print(len(list_data_muestra))

list_data_muestra = list_mano_verde_mano
filename = 'list_mano_verde_mano.txt'
print(len(list_data_muestra))

#with open(filename, 'w') as f:
#    f.write(json.dumps(list_data_muestra))


714


In [ ]:
os.chdir('/content/drive/My Drive/Mentoria/crop') 
! ls *.txt

list_mano_amarillo_amarillo.txt  list_mano_verde_mano.txt
list_mano_amarillo_mano.txt	 list_mano_verde_verde.txt


In [ ]:
filename = 'list_mano_verde_verde.txt'
#abrimos el archivo con la lista:
with open(filename, 'r') as f:
    list_jpg_muestra = json.loads(f.read())

print(len(list_jpg_muestra))

703


Cuando separamos las que son manos, se supone que las que no están en la lista, son guantes, y viceversa. De todas formas, lo contabilizamos como para verificarlo.

In [ ]:
# A filename_chosen lo vamos cammbiando, dependiendo del csv. 
#filename_mano_guante_ch = 'all_Mano-Amarillo_cropped_df.csv' 
filename_mano_guante_ch = 'all_Mano-Verde_cropped_df.csv' 
filename_mano_guante_df=pd.read_csv(filename_mano_guante_ch, index_col=0)
print(filename_mano_guante_df.head(2))
print(filename_mano_guante_df.shape)

                                       filename  xmin  ...  guantes  color
0    T_P_Trabajando_Mano-Verde_5_single_box.jpg  1215  ...       no  Verde
1  T_P_Trabajando_Mano-Verde_14_double_box1.jpg  1336  ...       no  Verde

[2 rows x 13 columns]
(1419, 13)


In [ ]:
#separamos las manos del archivo csv y corregimos el 'color':
color = 'mano'
#acá separamos las filas de manos de los archivos con guantes verdes/amarillos
value_list = list_jpg_muestra #list_mano_amarillo_mano
#value_list = list_mano_verde_mano
mano_si = filename_mano_guante_df.filename.isin(value_list)
mano_color_df = filename_mano_guante_df[mano_si].copy()
#generamos nuevas columnas, que utilizaremos mas adelante para identificar los distintos archivos:
mano_color_df['color'] = color
print(mano_color_df.shape)
mano_color_df.head(3)

(714, 13)


,filename,xmin,xmax,ymin,ymax,Time,Size,Type,Hand_Pixel_Width,Hand_Pixel_Height,Area,guantes,color
1,T_P_Trabajando_Mano-Verde_14_double_box1.jpg,1336,1396,252,292,T,P,train,60,40,2400,no,mano
5,T_P_Trabajando_Mano-Verde_17_double_box2.jpg,1370,1424,248,303,T,P,train,54,55,2970,no,mano
7,T_P_Trabajando_Mano-Verde_29_double_box2.jpg,1287,1335,211,255,T,P,train,48,44,2112,no,mano


In [ ]:
#guardamos la tabla en archivo csv
#name_mano_csv =  'Mano-Amarillo_'+ color +'_cropped_df.csv'
name_mano_csv =  'Mano-Verde_'+ color +'_cropped_df.csv'
#mano_color_df.to_csv(name_mano_csv) 
print(mano_color_df.shape)

(714, 13)


In [ ]:
! ls *.csv

all_Amarillos_cropped_df.csv	  all_Verdes_cropped_df.csv
all_df.csv			  Mano-Amarillo_amarillo_cropped_df.csv
all_Mano-Amarillo_cropped_df.csv  Mano-Amarillo_mano_cropped_df.csv
all_Manos_cropped_df.csv	  Mano-Verde_mano_cropped_df.csv
all_Mano-Verde_cropped_df.csv	  Mano-Verde_verde_cropped_df.csv


In [ ]:
#separamos las manos del archivo csv y corregimos el 'guantes':
guantes = 'si'
#acá separamos las filas de manos de los archivos con guantes verdes/amarillos
value_list = list_jpg_muestra  #list_mano_amarillo_mano
#value_list = list_mano_verde_mano
guante_si = ~filename_mano_guante_df.filename.isin(value_list)
guante_color_df = filename_mano_guante_df[guante_si].copy()
#generamos nuevas columnas, que utilizaremos mas adelante para identificar los distintos archivos:
guante_color_df['guantes'] = guantes
print(guante_color_df.shape)
guante_color_df.head(3)

(705, 13)


,filename,xmin,xmax,ymin,ymax,Time,Size,Type,Hand_Pixel_Width,Hand_Pixel_Height,Area,guantes,color
0,T_P_Trabajando_Mano-Verde_5_single_box.jpg,1215,1252,212,245,T,P,train,37,33,1221,si,Verde
2,T_P_Trabajando_Mano-Verde_14_double_box2.jpg,1235,1300,249,295,T,P,train,65,46,2990,si,Verde
3,T_P_Trabajando_Mano-Verde_7_single_box.jpg,1205,1251,199,259,T,P,train,46,60,2760,si,Verde


In [ ]:
#lo guardamos en archivo csv: 
#name_guante_csv =  'Mano-Amarillo_amarillo_cropped_df.csv'
name_guante_csv =  'Mano-Verde_verde_cropped_df.csv'
guante_color_df.to_csv(name_guante_csv) 
print(guante_color_df.shape)

(705, 13)


In [ ]:
# A filename_chosen lo vamos cammbiando, dependiendo del csv. 
filename_mano_guante_ch = 'Mano-Verde_verde_cropped_df.csv' 
filename_mano_guante_df=pd.read_csv(filename_mano_guante_ch, index_col=0)
print(filename_mano_guante_df.shape)
print(filename_mano_guante_df.head(2))


(705, 13)
                                       filename  xmin  ...  guantes  color
0    T_P_Trabajando_Mano-Verde_5_single_box.jpg  1215  ...       si  Verde
2  T_P_Trabajando_Mano-Verde_14_double_box2.jpg  1235  ...       si  Verde

[2 rows x 13 columns]


In [ ]:
row1 = filename_mano_guante_df.loc[filename_mano_guante_df['filename'] == 'N_P_Trabajando_Mano-Verde_69_double_box1.jpg']
row2 = filename_mano_guante_df.loc[filename_mano_guante_df['filename'] == 'N_P_Trabajando_Mano-Verde_69_double_box2.jpg']

In [ ]:
filename_mano_guante_df = filename_mano_guante_df[filename_mano_guante_df['filename'] != 'N_P_Trabajando_Mano-Verde_69_double_box1.jpg']
filename_mano_guante_df = filename_mano_guante_df[filename_mano_guante_df['filename'] != 'N_P_Trabajando_Mano-Verde_69_double_box2.jpg']
filename_mano_guante_df.shape


(703, 13)

In [ ]:
#name_guante_csv =  'Mano-Amarillo_amarillo_cropped_df.csv'
name_guante_csv =  'Mano-Verde_verde_cropped_df.csv'
filename_mano_guante_df.to_csv(name_guante_csv) 
print(filename_mano_guante_df.shape)

(703, 13)


In [ ]:
row1 = filename_mano_guante_df.loc[filename_mano_guante_df['filename'] == 'N_P_Trabajando_Mano-Verde_69_double_box1.jpg']
row2 = filename_mano_guante_df.loc[filename_mano_guante_df['filename'] == 'N_P_Trabajando_Mano-Verde_69_double_box2.jpg']

In [ ]:
! ls *.csv

all_Amarillos_cropped_df.csv	  all_Verdes_cropped_df.csv
all_df.csv			  Mano-Amarillo_amarillo_cropped_df.csv
all_Mano-Amarillo_cropped_df.csv  Mano-Amarillo_mano_cropped_df.csv
all_Manos_cropped_df.csv	  Mano-Verde_mano_cropped_df.csv
all_Mano-Verde_cropped_df.csv	  Mano-Verde_verde_cropped_df.csv


In [ ]:
# aquí haremos una chanchada... si se les ocurre una mejor forma, agradecida de no hacer enchastre.. esas 2 filas, 
# son las que necesita el archivo 'Mano-Amarillo_cropped_amarillo_df.csv' así que.... se las sumaremos! :D

# A filename_chosen lo vamos cammbiando, dependiendo del csv. 
filename_yellow = 'Mano-Amarillo_amarillo_cropped_df.csv' 
filename_yellow_df=pd.read_csv(filename_yellow, index_col=0)
filename_yellow_df = filename_yellow_df.append(row1)
filename_yellow_df = filename_yellow_df.append(row2)
print(filename_yellow_df.shape)
print(filename_yellow_df.head(2))


(736, 13)
                                          filename  xmin  ...  guantes     color
1  T_P_Trabajando_Mano-Amarillo_12_double_box2.jpg  1182  ...       si  Amarillo
2  T_P_Trabajando_Mano-Amarillo_52_double_box1.jpg  1152  ...       si  Amarillo

[2 rows x 13 columns]


In [ ]:
filename_yellow_df.tail()

,filename,xmin,xmax,ymin,ymax,Time,Size,Type,Hand_Pixel_Width,Hand_Pixel_Height,Area,guantes,color
1436,N_G_Mostrando_Mano-Amarillo_47_double_box2.jpg,1058,1243,371,537,N,G,test,185,166,30710,si,Amarillo
1437,N_G_Mostrando_Mano-Amarillo_44_double_box1.jpg,1106,1271,338,529,N,G,test,165,191,31515,si,Amarillo
1439,N_G_Mostrando_Mano-Amarillo_42_double_box1.jpg,1197,1331,334,610,N,G,test,134,276,36984,si,Amarillo
1203,N_P_Trabajando_Mano-Verde_69_double_box1.jpg,962,1000,221,251,N,P,test,38,30,1140,si,Verde
1204,N_P_Trabajando_Mano-Verde_69_double_box2.jpg,936,979,243,275,N,P,test,43,32,1376,si,Verde


In [ ]:
#lo guardamos en archivo csv: 
#name_guante_csv =  'Mano-Amarillo_amarillo_cropped_df.csv'
#filename_yellow_df.to_csv(name_guante_csv) 
#print(filename_yellow_df.shape)

(736, 13)


In [ ]:
list_green_csv = filename_mano_guante_df.filename.unique().tolist()

In [ ]:
#verificamos que haya la misma cantidad de filas únicas como fotos en el dataset
diff(list_jpg_muestra, list_green_csv)

['N_P_Trabajando_Mano-Verde_69_double_box1.jpg',
 'N_P_Trabajando_Mano-Verde_69_double_box2.jpg']

In [ ]:
# 'Mano-Amarillo_cropped_mano_df.csv'      # shape : (707, 13)
# 'Mano-Amarillo_cropped_amarillo_df.csv'  # shape : (734, 13)
# ['N_P_Trabajando_Mano-Verde_69_double_box1.jpg',  'N_P_Trabajando_Mano-Verde_69_double_box2.jpg'] están en la lista de Verdes pero
# son dos guantes amarillos!
# 'Mano-Verde_cropped_mano_df.csv'         # shape : (86, 13)
# 'Mano-Verde_cropped_verde_df.csv'        # shape : (83, 13)


#  guardamos una copia del csv en sus carpetas correspondientes

Bueeeeno... luego de una magia anterior, tenemos la mismas fotos en las carpetas y en los archivos csv. Ahora podemos terminar de armar el dataset:

In [ ]:
#volvemos a leer los archivos:
file_csv_m_g = ['Mano-Amarillo_mano_cropped_df.csv', 'Mano-Amarillo_amarillo_cropped_df.csv',
         'Mano-Verde_mano_cropped_df.csv', 'Mano-Verde_verde_cropped_df.csv']

for file_csv in file_csv_m_g:
  print(file_csv)
  
  cropped_df = pd.read_csv(file_csv, index_col=0)
  list_filename_df = cropped_df.filename.unique().tolist()

  print('cantidad de fotos: ', len(list_filename_df)) 
  print(cropped_name_df,'shape:', cropped_df.shape, '\n')

Mano-Amarillo_mano_cropped_df.csv
cantidad de fotos:  707
all_Verdes_cropped_df.csv shape: (707, 13) 

Mano-Amarillo_amarillo_cropped_df.csv
cantidad de fotos:  736
all_Verdes_cropped_df.csv shape: (736, 13) 

Mano-Verde_mano_cropped_df.csv
cantidad de fotos:  714
all_Verdes_cropped_df.csv shape: (714, 13) 

Mano-Verde_verde_cropped_df.csv
cantidad de fotos:  703
all_Verdes_cropped_df.csv shape: (703, 13) 



In [ ]:
# os.chdir('/content/drive/My Drive/Mentoria/crop/Mano-Amarillo/Mano')        # 707 fotos
# os.chdir('/content/drive/My Drive/Mentoria/crop/Mano-Amarillo/Amarillo')     # 736 fotos  

# os.chdir('/content/drive/My Drive/Mentoria/crop/Mano-Verde/Mano')           # 714 fotos
# os.chdir('/content/drive/My Drive/Mentoria/crop/Mano-Verde/Verde')          # 703 fotos

Lo siguiente podríamos abordarlo juntando todas las manos y guantes. Pero para mayor control y no enloquecernos, vamos carpeta por carpeta, generando los vectores de features y luego los juntamos.

Si se animan y quieren automatizar lo siguiente, genial :)

In [ ]:
#volvemos a la carpeta classes, vemos de copiar todos los csv correspondientes a esta carpeta
os.chdir('/content/drive/My Drive/Mentoria/crop')
#listamos solo las carpetas
! ls -d */
! ls -d */*/

Amarillos/  Mano-Amarillo/  Manos/  Mano-Verde/  Verdes/
Mano-Amarillo/Amarillo/  Mano-Verde/Mano/
Mano-Amarillo/Mano/	 Mano-Verde/Verde/


In [ ]:
#listamos los csv
extension = 'csv'
list_files_csv = [i for i in glob.glob('*_cropped_df.{}'.format(extension))]
list_files_csv = sorted(list_files_csv, reverse=False)
print(list_files_csv)

['Amarillos_cropped_df.csv', 'Mano-Amarillo_amarillo_cropped_df.csv', 'Mano-Amarillo_mano_cropped_df.csv', 'Mano-Verde_mano_cropped_df.csv', 'Mano-Verde_verde_cropped_df.csv', 'Manos_cropped_df.csv', 'Verdes_cropped_df.csv']


In [ ]:
#volvemos a leer los archivos:
tipos = list_files_csv

for tipo in tipos:
  print(tipo)
  cropped_df = pd.read_csv(tipo, index_col=0)
  list_filename_df = cropped_df.filename.unique().tolist()

  print('cantidad de fotos: ', len(list_filename_df)) 
  print(tipo,'shape:', cropped_df.shape, '\n')

Amarillos_cropped_df.csv
cantidad de fotos:  3958
Amarillos_cropped_df.csv shape: (3958, 13) 

Mano-Amarillo_amarillo_cropped_df.csv
cantidad de fotos:  736
Mano-Amarillo_amarillo_cropped_df.csv shape: (736, 13) 

Mano-Amarillo_mano_cropped_df.csv
cantidad de fotos:  707
Mano-Amarillo_mano_cropped_df.csv shape: (707, 13) 

Mano-Verde_mano_cropped_df.csv
cantidad de fotos:  714
Mano-Verde_mano_cropped_df.csv shape: (714, 13) 

Mano-Verde_verde_cropped_df.csv
cantidad de fotos:  703
Mano-Verde_verde_cropped_df.csv shape: (703, 13) 

Manos_cropped_df.csv
cantidad de fotos:  4524
Manos_cropped_df.csv shape: (4524, 13) 

Verdes_cropped_df.csv
cantidad de fotos:  3855
Verdes_cropped_df.csv shape: (3855, 13) 



In [ ]:
# acá vamos cambiando los nombres de las carpetas.
# [Amarillos/  Mano-Amarillo/  Manos/  Mano-Verde/  Verdes/]
#! python3 features.py Mano-Verde/Verde/

In [ ]:
# copiamos el archivo json generado
#! mv features.json features_Mano-Verde_verde_cropped.json

In [ ]:
#listamos los json
extension = 'json'
list_files_json = [i for i in glob.glob('*.{}'.format(extension))]
list_files_json = sorted(list_files_json, reverse=False)
list_files_json

['features_Amarillos_cropped.json',
 'features_Mano-Amarillo_amarillo_cropped.json',
 'features_Mano-Amarillo_mano_cropped.json',
 'features_Mano-Verde_mano_cropped.json',
 'features_Mano-Verde_verde_cropped.json',
 'features_Manos_cropped.json',
 'features_Verdes_cropped.json']

In [ ]:
#Leemos y ordenamos por nombre los archivos csv:
for csv_df in list_files_csv:
  cropped_df = pd.read_csv(csv_df, index_col=0).sort_values(by=['filename'])
  print(csv_df, ':', cropped_df.shape)
  

Amarillos_cropped_df.csv : (3958, 13)
Mano-Amarillo_amarillo_cropped_df.csv : (736, 13)
Mano-Amarillo_mano_cropped_df.csv : (707, 13)
Mano-Verde_mano_cropped_df.csv : (714, 13)
Mano-Verde_verde_cropped_df.csv : (703, 13)
Manos_cropped_df.csv : (4524, 13)
Verdes_cropped_df.csv : (3855, 13)


In [ ]:
#acá armamos todos los archivos con los features de cada una de las carpetas:
for json_file in list_files_json:
  json_csv_name = json_file[:-5]+'_json_df.csv'
  print(json_csv_name)

  df = pd.read_json(json_file)
  df_json_transpose = df.T.reset_index().copy()
  #print(df_json_transpose.shape)

  #para listar solo los nombres de los '.jpg'
  list_index_json = df_json_transpose['index']
  #print(len(list_index_json))

  index_json = []

  for file_jpg in list_index_json:
    x = re.search('[A-Z]\w+_.+jpg', file_jpg)
    index_json.append(x.group())

  #print(len(index_json)) 

  df_json_transpose['filename'] = index_json
  df_json_transpose = df_json_transpose.drop(['index'], axis=1)
  df_json_transpose = df_json_transpose.set_index('filename')
  df_json_transpose = df_json_transpose.reset_index().sort_values(by=['filename']).copy()
  print(df_json_transpose.shape)
  df_json_transpose.to_csv(json_csv_name)

features_Amarillos_cropped_json_df.csv
(3958, 2049)
features_Mano-Amarillo_amarillo_cropped_json_df.csv
(736, 2049)
features_Mano-Amarillo_mano_cropped_json_df.csv
(707, 2049)
features_Mano-Verde_mano_cropped_json_df.csv
(714, 2049)
features_Mano-Verde_verde_cropped_json_df.csv
(703, 2049)
features_Manos_cropped_json_df.csv
(4524, 2049)
features_Verdes_cropped_json_df.csv
(3855, 2049)


In [ ]:
#listamos los json
extension = 'csv'
list_files_json_csv = [i for i in glob.glob('*cropped_json_df.{}'.format(extension))]
list_files_json_csv = sorted(list_files_json_csv, reverse=False)
list_files_json_csv

['features_Amarillos_cropped_json_df.csv',
 'features_Mano-Amarillo_amarillo_cropped_json_df.csv',
 'features_Mano-Amarillo_mano_cropped_json_df.csv',
 'features_Mano-Verde_mano_cropped_json_df.csv',
 'features_Mano-Verde_verde_cropped_json_df.csv',
 'features_Manos_cropped_json_df.csv',
 'features_Verdes_cropped_json_df.csv']

In [ ]:
#juntamos el csv de features con el generado de los json
for csv_df, json_csv_file in zip(list_files_csv, list_files_json_csv):
  left  = pd.read_csv(csv_df, index_col=0).sort_values(by=['filename'])
  right = pd.read_csv(json_csv_file, index_col=0).sort_values(by=['filename'])
  print('left', csv_df, ':', left.shape)
  print('right', json_csv_file, ':', right.shape)
  result = pd.merge(left, right, on='filename')
  result_name_df = result.copy()
  print(result_name_df.shape)
  result_name = csv_df[:-7] + '_features_labels_df.csv'
  result_name_df.to_csv(result_name) 
  

left Amarillos_cropped_df.csv : (3958, 13)
right features_Amarillos_cropped_json_df.csv : (3958, 2049)
(3958, 2061)
left Mano-Amarillo_amarillo_cropped_df.csv : (736, 13)
right features_Mano-Amarillo_amarillo_cropped_json_df.csv : (736, 2049)
(736, 2061)
left Mano-Amarillo_mano_cropped_df.csv : (707, 13)
right features_Mano-Amarillo_mano_cropped_json_df.csv : (707, 2049)
(707, 2061)
left Mano-Verde_mano_cropped_df.csv : (714, 13)
right features_Mano-Verde_mano_cropped_json_df.csv : (714, 2049)
(714, 2061)
left Mano-Verde_verde_cropped_df.csv : (703, 13)
right features_Mano-Verde_verde_cropped_json_df.csv : (703, 2049)
(703, 2061)
left Manos_cropped_df.csv : (4524, 13)
right features_Manos_cropped_json_df.csv : (4524, 2049)
(4524, 2061)
left Verdes_cropped_df.csv : (3855, 13)
right features_Verdes_cropped_json_df.csv : (3855, 2049)
(3855, 2061)


In [ ]:
#listamos los csv
extension = 'csv'
list_files_features_csv = [i for i in glob.glob('*features_labels_df.{}'.format(extension))]
list_files_features_csv = sorted(list_files_features_csv, reverse=False)
print(list_files_features_csv)
print(len(list_files_features_csv))

['Amarillos_cropped_features_labels_df.csv', 'Mano-Amarillo_amarillo_cropped_features_labels_df.csv', 'Mano-Amarillo_mano_cropped_features_labels_df.csv', 'Mano-Verde_mano_cropped_features_labels_df.csv', 'Mano-Verde_verde_cropped_features_labels_df.csv', 'Manos_cropped_features_labels_df.csv', 'Verdes_cropped_features_labels_df.csv']
7


In [ ]:
#juntamos todos los arhivos en una hermosa y gigante tabla final!! 
csv_wlabels = (pd.read_csv(f, index_col=0) for f in list_files_features_csv)
csv_wlabels_all_df = pd.concat(csv_wlabels, ignore_index=True)
print(csv_wlabels_all_df.shape)
csv_wlabels_all_df.head()
#csv_wlabels_all_df.to_csv('csv_wlabels_all_df.csv') 

(15197, 2061)


In [ ]:
#leemos el archivo de la tabla:
csv_wlabels_df = pd.read_csv('csv_wlabels_all_df.csv', index_col=0)
print(csv_wlabels_df.shape)
csv_wlabels_df.head()

(15197, 2061)


,filename,xmin,xmax,ymin,ymax,Time,Size,Type,Hand_Pixel_Width,Hand_Pixel_Height,Area,guantes,color,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,M_G_Mostrando_Amarillos_0_double_box1.jpg,218,461,464,715,M,G,train,243,251,60993,si,Amarillo,0.740974,1.066019,0.132695,0.224892,0.140456,0.129134,0.201447,1.076733,0.461108,0.299800,0.457940,0.201904,0.569677,0.737381,0.480929,0.200076,0.018410,0.674050,0.028826,0.005310,0.128693,0.592353,0.138541,0.141684,0.363294,0.264674,0.519292,...,0.463567,0.091676,0.300955,0.696112,2.409413,0.536980,0.102273,0.157441,0.165093,1.067619,0.751934,0.231443,0.487437,0.446320,0.822407,0.527523,0.217110,0.489194,1.535306,0.521390,0.599480,0.201182,0.391337,0.040206,1.347440,0.448389,0.462778,0.590751,0.877142,0.009986,0.939221,0.859877,0.981910,0.356366,1.748501,0.288037,0.079172,0.266234,0.078312,0.421612
1,M_G_Mostrando_Amarillos_0_double_box2.jpg,617,865,432,598,M,G,train,248,166,41168,si,Amarillo,1.621832,0.208859,0.627041,0.061717,0.052486,0.064176,0.479256,1.068516,0.509059,0.207385,0.218195,0.169190,0.695486,0.237641,0.146711,0.126334,0.029900,0.148363,0.126833,0.181351,0.044585,0.447270,0.134360,0.113650,1.153900,0.653832,0.041024,...,0.816892,0.027489,0.183116,0.626535,1.734605,0.672962,0.030057,0.101685,0.295544,0.392251,0.874054,0.105285,0.110713,0.421417,1.335695,0.500971,1.139208,0.110803,0.850265,0.618181,0.077352,0.376629,0.319234,0.783792,0.427963,0.133063,0.064981,0.017857,0.320706,0.219192,1.199712,0.808466,0.491182,0.356616,0.812572,0.481357,0.071471,0.572322,0.052072,0.213453
2,M_G_Mostrando_Amarillos_100_double_box1.jpg,831,1138,256,408,M,G,train,307,152,46664,si,Amarillo,1.029100,0.246964,0.825601,0.460009,0.467037,0.339540,0.653041,1.324649,0.890402,0.488183,0.639772,0.085819,0.928661,0.148958,0.066155,0.125391,0.191808,0.506981,0.257565,0.073959,0.115524,0.552251,0.764911,0.047104,0.207601,0.126297,0.286010,...,0.367434,0.436251,0.000000,0.863331,0.112969,0.902164,0.040710,0.467132,0.242757,0.853286,0.677067,0.300664,0.587456,0.112658,1.077973,0.144736,0.294853,0.065939,0.708062,1.192815,0.317312,0.421216,0.425882,1.091190,0.595093,0.138662,0.045023,0.543350,0.155850,0.305535,0.463963,0.355296,0.360880,0.322859,0.917611,0.408334,0.733588,0.416891,0.151471,0.045931
3,M_G_Mostrando_Amarillos_100_double_box2.jpg,511,812,8,228,M,G,train,301,220,66220,si,Amarillo,0.086092,0.455828,0.418580,0.131673,0.437982,0.424093,0.350489,1.974670,0.183887,0.241193,0.046988,0.312450,0.497026,0.135973,0.195146,0.010869,0.109539,0.234128,0.235778,0.108441,0.054660,0.327069,0.439913,0.050670,0.159292,0.159228,0.311495,...,0.625021,0.219692,0.029219,0.496582,1.540741,2.338270,0.039844,0.174714,0.383931,0.449525,0.104648,0.205243,0.104140,0.109657,1.092486,0.087052,0.451129,0.034648,0.272219,0.209507,0.138235,0.193764,0.081227,1.006930,1.506502,0.044992,1.047619,0.605296,0.391379,0.164156,0.325708,0.028549,0.225465,0.325055,0.460382,0.127341,0.140412,0.157423,0.296102,0.023314
4,M_G_Mostrando_Amarillos_101_double_box1.jpg,668,988,0,222,M,G,train,320,222,71040,si,Amarillo,0.068881,0.185060,0.427007,0.637546,0.357156,0.125816,0.450526,1.246901,0.236921,0.393104,0.632759,0.413947,0.119917,0.192305,0.372677,0.075668,0.103642,0.165934,0.239047,0.054987,0.129182,0.178563,0.217686,0.130172,0.080214,0.346873,0.392784,...,0.205059,0.225329,0.062677,0.558553,1.780167,1.919532,0.165511,0.649451,0.168220,0.127490,0.310012,0.071468,0.196005,0.042177,1.261848,0.216220,0.501204,0.194764,0.183244,0.426975,0.477355,0.349516,0.131819,0.489718,0.981365,0.049099,0.471963,0.190929,0.413278,0.175050,0.523058,0.028629,0.076427,0.462088,0.694069,0.060703,0.065414,0.296513,0.368063,0.039469
